In [ ]:
! cd ~/; git clone https://github.com/alvelvis/ACDC-UD; git clone https://github.com/comcorhd/BIG-Oil-NLP

In [5]:
import sys
sys.path.append('/home/elvis/ACDC-UD')
import estrutura_ud
import os
import re

# Converter 400 TEM para utf-8
Necessário ter a pasta do Petrolês em utf-16 (no repositório a conversão para utf-8 já foi realizada)

Etapa necessária, caso contrário o UDPipe terá problemas

In [ ]:
! mkdir ~/Petrolês/processado-utf8
! mkdir ~/Petrolês/gambiarra-utf8
for file in os.listdir("~/Petrolês/TXT_Teses e Monografias"):
    if file.endswith(".txt"):
        with open(f"~/Petrolês/TXT_Teses e Monografias/{file}", encoding="utf-16") as f:
            with open(f"~/Petrolês/processado-utf8/{file}", 'w', encoding="utf-8") as r:
                r.write(f.read())
                
for file in os.listdir("~/Petrolês/Corpus Gambiarra"):
    if file.endswith(".txt"):
        with open(f"~/Petrolês/Corpus Gambiarra/{file}", encoding="utf-16") as f:
            with open(f"~/Petrolês/gambiarra-utf8/{file}", 'w', encoding="utf-8") as r:
                r.write(f.read())             

# Anotar 400TEM
--> DEMORADO

In [ ]:
! ~/BIG-Oil-NLP/udpipe/udpipe-1.2.0 --tokenize --tag --parse ~/BIG-Oil-NLP/udpipe/bosqueud_2.5_workbench.udpipe ~/Petrolês/processado-utf8/*.txt > ~/Petrolês/bem_processado.conllu

Loading UDPipe model: done.


In [ ]:
! ~/BIG-Oil-NLP/udpipe/udpipe-1.2.0 --tokenize --tag --parse ~/BIG-Oil-NLP/udpipe/bosqueud_2.5_workbench.udpipe ~/Petrolês/gambiarra-utf8/*.txt > ~/Petrolês/gambiarra.conllu

Loading UDPipe model: done.


# Adicionar sent_id às frases das 400TEM

In [ ]:
! python3 ~/BIG-Oil-NLP/scripts/fix_sent_id.py ~/Petrolês/bem_processado_udpipe.conllu > ~/Petrolês/bem_processado_sentid.conllu

In [ ]:
! python3 ~/BIG-Oil-NLP/scripts/fix_sent_id.py ~/Petrolês/gambiarra.conllu > ~/Petrolês/gambiarra_sentid.conllu

# Corrigir fórmulas nas 400TEM
(ou só gambiarra)

In [ ]:
with open("~/Petrolês/gambiarra_sentid.conllu") as f:
    gambiarra = f.read()

gambiarra_split = gambiarra.split("\n\n")
print(len(gambiarra_split))
gambiarra = [x for x in gambiarra_split if not "\t\n" in x and not re.search(r'\n[^\d#]', x)]
print(len(gambiarra))

with open("~/Petrolês/gambiarra_sentid_formula.conllu", 'w') as f:
    f.write("\n\n".join(gambiarra))

# Abrir 400TEM
--> DEMORADO

In [ ]:
gambiarra = estrutura_ud.Corpus()
gambiarra.load("~/Petrolês/gambiarra_sentid_formula.conllu")

In [ ]:
bemprocessado = estrutura_ud.Corpus()
bemprocessado.load("~/Petrolês/bem_processado_sentid.conllu")

# Teste de documentos em bemprocessado e gambiarra 
(estranho -- número diferente. Anotar novamente.)

In [ ]:
lista_documentos_bemprocessado = []
[lista_documentos_bemprocessado.append(x.rsplit("-", 1)[0]) for x in bemprocessado.sentences if x.rsplit("-", 1)[0] not in lista_documentos_bemprocessado]

lista_documentos_gambiarra = []
[lista_documentos_gambiarra.append(x.rsplit("-", 1)[0]) for x in gambiarra.sentences if x.rsplit("-", 1)[0] not in lista_documentos_gambiarra]

fora_de_gambiarra = [x for x in lista_documentos_bemprocessado if x not in lista_documentos_gambiarra]
fora_de_bemprocessado = [x for x in lista_documentos_gambiarra if x not in lista_documentos_bemprocessado]
print(f"Documentos em bemprocessado ({len(lista_documentos_bemprocessado)}) que faltam em gambiarra ({len(lista_documentos_gambiarra)}):")
print(fora_de_gambiarra)
print("\nDocumentos em gambiarra que faltam em bemprocessado:")
print(fora_de_bemprocessado)

# Normalização dos documentos em ambos arquivos

In [ ]:
lista_documentos_proibidos = fora_de_gambiarra + fora_de_bemprocessado
print("Documentos proibidos:")
print(lista_documentos_proibidos)
gambiarra.sentences = {x: y for x, y in gambiarra.sentences.items() if x.rsplit("-", 1)[0] not in lista_documentos_proibidos}
bemprocessados.sentences = {x: y for x, y in bemprocessados.sentences.items() if x.rsplit("-", 1)[0] not in lista_documentos_proibidos}

fora_de_gambiarra = [x for x in lista_documentos_bemprocessado if x not in lista_documentos_gambiarra]
fora_de_bemprocessado = [x for x in lista_documentos_gambiarra if x not in lista_documentos_bemprocessado]
print(f"Documentos em bemprocessado ({len(lista_documentos_bemprocessado)}) que faltam em gambiarra ({len(lista_documentos_gambiarra)}):")
print(fora_de_gambiarra)
print("\nDocumentos em gambiarra que faltam em bemprocessado:")
print(fora_de_bemprocessado)

# Anotar com jPTDP
--> DEMORADO

In [ ]:
! sudo -S apt install python-pip --yes < ~/sudopass
! pip install cython numpy
! pip install dynet==2.0.3
! cd ~/BIG-Oil-NLP/jPTDP; wget https://www.dropbox.com/s/fn0r48xhn67inpt/outputs.tar.gz; tar -xvzf outputs.tar.gz

In [ ]:
! python ~/BIG-Oil-NLP/jPTDP/src/jPTDP.py --predict --model ~/BIG-Oil-NLP/jPTDP/outputs/jPTDP_pt_ud.model --params ~/BIG-Oil-NLP/jPTDP/outputs/jPTDP_pt_ud.params --test ~/Petrolês/gambiarra_sentid_formula.conllu --outdir ~/Petrolês/ --output ~/Petrolês/gambiarra_jptdp.conllu

In [ ]:
! python ~/BIG-Oil-NLP/jPTDP/src/jPTDP.py --predict --model ~/BIG-Oil-NLP/jPTDP/outputs/jPTDP_pt_ud.model --params ~/BIG-Oil-NLP/jPTDP/outputs/jPTDP_pt_ud.params --test ~/Petrolês/bemprocessado_sentid.conllu --outdir ~/Petrolês/ --output ~/Petrolês/bemprocessado_jptdp.conllu

# Abrir 400TEM
--> DEMORADO

In [ ]:
gambiarra = estrutura_ud.Corpus()
gambiarra.load("~/Petrolês/gambiarra_jptdp.conllu")

In [ ]:
bemprocessado = estrutura_ud.Corpus()
bemprocessado.load("~/Petrolês/bemprocessado_jptdp.conllu")

# Volte três casas: teste e normalização

# Distribuição de DP e POS (ajustar onde)

In [ ]:
dic_pos_dp_gambiarra = {}
dic_pos_dp_bemprocessado = {}
for sent, sentence in gambiarra.sentences.items():
    for t, token in enumerate(sentence.tokens):
        if not '-' in token.id:
            dic_pos_dp_gambiarra[token.upos] = 1 if not token.upos in dic_pos_dp_gambiarra else dic_pos_dp_gambiarra[token.upos] + 1
            dic_pos_dp_gambiarra[token.deprel] = 1 if not token.deprel in dic_pos_dp_gambiarra else dic_pos_dp_gambiarra[token.deprel] + 1
for sent, sentence in bemprocessado.sentences.items():
    for t, token in enumerate(sentence.tokens):
        if not '-' in token.id:
            dic_pos_dp_bemprocessado[token.upos] = 1 if not token.upos in dic_pos_dp_bemprocessado else dic_pos_dp_bemprocessado[token.upos] + 1
            dic_pos_dp_bemprocessado[token.deprel] = 1 if not token.deprel in dic_pos_dp_bemprocessado else dic_pos_dp_bemprocessado[token.deprel] + 1

print(f"Classes em gambiarra ({len(dic_pos_dp_gambiarra)})")
print(sorted(dic_pos_dp_gambiarra))
print(f"\nClasses em bemprocessado ({len(dic_pos_dp_bemprocessado)})")
print(sorted(dic_pos_dp_bemprocessado))

# Número de sentenças e tokens

In [ ]:
documentos = []
[documentos.append(x.rsplit("-", 1)[0]) for x in bemprocessado.sentences if x.rsplit("-", 1)[0] not in documentos]
print(f"{len(documentos)} documentos foram encontrados em FINAL")
print("\n".join(["* " + x for x in documentos]))

# a contagem de tokens não leva em consideração aqueles que têm "-" no seu id (indicação de contração (a contração em si conta, é claro))
print("\n|Versão|Sentenças|Tokens|")
print("|---|---|---|")
tokens_gambiarra = len([x for sentence in gambiarra.sentences.values() for x in sentence.tokens if not '-' in x.id])
tokens_final = len([x for sentence in bemprocessado.sentences.values() for x in sentence.tokens if not '-' in x.id])
print("|GAMBIARRA|{}|{}|".format(len(gambiarra.sentences), tokens_gambiarra))
print("|FINAL|{}|{}|".format(len(bemprocessado.sentences), tokens_final))

# Distribuição das categorias bruta

In [ ]:
print("\nDistribuição das categorias bruta")
print("| {:25} | {:25} | {:25} |".format('Anotação', 'GAMBIARRA', 'FINAL'))
print("| {:25} | {:25} | {:25} |".format("---", "---", "---"))
lista_pos_dp = [x for x in dic_pos_dp_gambiarra]
[lista_pos_dp.append(x) for x in dic_pos_dp_bemprocessado if not x in lista_pos_dp]
for pos_dp in sorted(lista_pos_dp):
    print("| {:25} | {:25} | {:25} |".format(pos_dp, dic_pos_dp_gambiarra[pos_dp] if pos_dp in dic_pos_dp_gambiarra else 0, dic_pos_dp_bemprocessado[pos_dp] if pos_dp in dic_pos_dp_bemprocessado else 0))

# Distribuição das categorias relativa ao total de tokens da versão

In [ ]:
print("\nDistribuição das categorias relativa ao total de tokens na versão")
print("| {:25} | {:25} | {:25} |".format('Anotação', 'GAMBIARRA', 'FINAL'))
print("| {:25} | {:25} | {:25} |".format("---", "---", "---"))
lista_pos_dp = [x for x in dic_pos_dp_gambiarra]
[lista_pos_dp.append(x) for x in dic_pos_dp_bemprocessado if not x in lista_pos_dp]
for pos_dp in sorted(lista_pos_dp):
    print("| {:25} | {:25} | {:25} |".format(pos_dp, str((dic_pos_dp_gambiarra[pos_dp]/tokens_gambiarra)*100)+'%' if pos_dp in dic_pos_dp_gambiarra else 0, str((dic_pos_dp_bemprocessado[pos_dp]/tokens_final)*100)+'%' if pos_dp in dic_pos_dp_bemprocessado else 0))

# Lemas

In [ ]:
lista_lemas_gambiarra = [lista_lemas_gambiarra.append(x.lemma) for y in gambiarra.sentences.values() for x in y.tokens if not '-' in x.id]
lista_lemas_bemprocessado = [lista_lemas_bemprocessado.append(x.lemma) for y in bemprocessado.sentences.values() for x in y.tokens if not '-' in x.id]

print("|{:25}|{:25}|{:25}|".format('', 'GAMBIARRA', 'FINAL'))
print("|{:25}|{:25}|{:25}|".format("---", "---", "---"))
print("|{:25}|{:25}|{:25}|".format('Lemas diferentes', len(list(dict.fromkeys(lista_lemas_gambiarra))), len(list(dict.fromkeys(lista_lemas_bemprocessado)))))

In [ ]:
print("\nDistribuição dos lemas em GAMBIARRA")
print("|{:25}|{:25}|{:25}|".format('LEMA', 'OCORRÊNCIA', 'REPRESENTATIVIDADE'))
print("|{:25}|{:25}|{:25}|".format('', 'GAMBIARRA', 'FINAL'))
[
    print("|{:25}|{:25}|{:25}|".format(
            x, 
            lista_lemas_gambiarra.count(x), 
            str((lista_lemas_gambiarra.count(x)/len(lista_lemas_gambiarra))*100)+'%',
        ))
        for x in sorted(lista_lemas_gambiarra, reverse=True, key=lambda y: lista_lemas_gabiarra.count(y))
]